In [1]:
from selenium import webdriver as wd
import time
from bs4 import BeautifulSoup as bs
import pymysql as mysql

In [62]:
### url define
#CPU 594
#메인보드 595
#메모리 596
#그래픽카드 597
#SSD 604
#HDD 598
#케이스 600
#파워 601
### 

In [87]:
conn = mysql.connect(host='localhost',
                       user='root',
                       password='parkingC103!',
                       db='comssa',
                       charset='utf8')




In [147]:

def crawCard(urlNum, pageNum, parts, driver):
    baseUrl = 'https://shop.danawa.com/main/?controller=goods&methods=index&productRegisterAreaGroupSeq=78&serviceSectionSeq=%d#%d'
    crawUrl = baseUrl%(urlNum,pageNum)
    driver.get(crawUrl)
    time.sleep(1)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    crawlist1 = '#productListContainer > div > ul:nth-of-type(1) '
    title1 = crawlist1 + 'a.goodsLink'
    price1 = crawlist1 + 'div.price_info span.num'
    spec1 = crawlist1 + '> li:nth-child(%d) div.main_info > dl > dd > ul > li'
    titles1 = soup.select(title1)
    prices1 = soup.select(price1)


    crawlist2 = '#productListContainer > div > ul:nth-of-type(2) '
    title2 = crawlist2 + 'a.goodsLink'
    price2 = crawlist2 + 'div.price_info span.num'
    spec2 = crawlist2+'> li:nth-child(%d) div.main_info > dl > dd > ul > li'
    titles2 = soup.select(title2)
    prices2 = soup.select(price2)
    
    insertTitleList=[]
    insertSeqList=[]
    insertPriceList=[]
    insertSpecList=[]
    
    for n in range(len(titles1)):
        insertTitleList.append(titles1[n].text.strip())
        insertPriceList.append(prices1[n].text.strip())
        insertSeqList.append(titles1[n]['productseq'])
#         print(titles1[n].text.strip() +'        ' +prices1[n].text.strip())
#         print()
#         print(titles1[n]['productseq'])
        spec1s = spec1%(n+1)
        specs = soup.select(spec1s)
        speclist1=[]
        for m in specs:
            speclist1.append(m.text.strip())
        while len(speclist1)<20:
            speclist1.append('')
        speclistText="','".join(speclist1)
        insertSpecList.append(speclistText)
    for n in range(len(titles2)):
        insertTitleList.append(titles2[n].text.strip())
        insertPriceList.append(prices2[n].text.strip())
        insertSeqList.append(titles2[n]['productseq'])
        spec2s = spec2%(n+1)
        specs = soup.select(spec2s)
        speclist2=[]
        for m in specs:
            speclist2.append(m.text.strip())
        while len(speclist2)<20:
            speclist2.append('')
        speclistText="','".join(speclist2)
        print(len(speclist2))
        insertSpecList.append(speclistText)
        
    return insertTitleList,insertSpecList,insertPriceList,insertSeqList

In [160]:
driver = wd.Chrome('chromedriver.exe')

titles,specs,prices,seqs=crawCard(604,1,'memory',driver)
print(specs[0])
#594 cpu under 20
#595 mainboard max 58
#596 memory under 20
#597 gpu max 21
#598 hdd max 21
#600 case max 23
#601 power max 27
#604 ssd max 34

C:\Users\multicampus\AppData\Local\Temp\ipykernel_16776\3043758442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome('chromedriver.exe')


29
32
29
33
21
29
29
31
34
31
21
31
34
21
33
29
34
31
29
31
31
34
내장형SSD','M.2 (2280)','PCIe3.0x4 (32GT/s)','NVMe 1.3','TLC(토글)','3D낸드','컨트롤러:KIOXIA','[성능]','순차읽기:1,700MB/s','순차쓰기:1,600MB/s','읽기IOPS:최대 350K','쓰기IOPS:최대 400K','[지원기능]','TRIM','GC','SLC캐싱','S.M.A.R.T','ECC','전용 S/W','[환경특성]','MTBF:150만시간','TBW:200TB','A/S기간:5년','방열판 미포함','두께:2.23mm','6.9g


In [149]:
print(len(titles))
print(len(specs))
print(len(seqs))
print(len(prices))
print(specs[0])

32
32
32
32
내장형SSD','M.2 (2280)','PCIe3.0x4 (32GT/s)','NVMe 1.3','TLC(토글)','3D낸드','컨트롤러:KIOXIA','[성능]','순차읽기:1,700MB/s','순차쓰기:1,600MB/s','읽기IOPS:최대 350K','쓰기IOPS:최대 400K','[지원기능]','TRIM','GC','SLC캐싱','S.M.A.R.T','ECC','전용 S/W','[환경특성]','MTBF:150만시간','TBW:200TB','A/S기간:5년','방열판 미포함','두께:2.23mm','6.9g


In [137]:
conn = mysql.connect(host='localhost',
                       user='root',
                       password='parkingC103!',
                       db='comssa',
                       charset='utf8')


with conn:
    with conn.cursor() as cur:
        for i in range(len(titles)):
            sql = "INSERT INTO tb_cpu VALUES (%s, %s,%s,%s)"%(\
            seqs[i],\
            "'%s'"%titles[i]\
            ,prices[i].replace(',', ''),\
            "'%s'"%specs[i]\
            )
            cur.execute(sql)
        conn.commit()

INSERT INTO tb_cpu VALUES (17913311, 'AMD 라이젠5-5세대 7600X (라파엘) (정품)',502180,'AMD(소켓AM5)','5세대(Zen4)','5nm','6코어','12쓰레드','기본 클럭:4.7GHz','최대 클럭:5.3GHz','L3 캐시:32MB','TDP:105W','PCIe5.0','메모리 규격:DDR5','5200MHz','내장그래픽:탑재','AMD 라데온 그래픽','쿨러:미포함','','','','','')


In [11]:
options = wd.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = wd.Chrome('chromedriver.exe')#,chrome_options=options)
driver.get('https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+7950X')
time.sleep(1)
html = driver.page_source
soup = bs(html, 'html.parser')

spec = soup.select('#main_content > div.wrapper > div.main-cmps > div:nth-child(2) > div.ov-scroll > div > div.desc-body > div.left-desc-cpu > p:nth-child(3)')

C:\Users\multicampus\AppData\Local\Temp\ipykernel_3300\3826102534.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome('chromedriver.exe')#,chrome_options=options)


In [41]:
spec = soup.select('#main_content div.left-desc-cpu > p:nth-child(4)')
specp = soup.select('#main_content div.left-desc-cpu > p:nth-child(4) > strong')

In [60]:
clock=specp[0].text.strip()
clockspeed=spec[0].text.strip()

clockspeed=clockspeed.replace(clock,'')
clockspeed=clockspeed.replace('GHz','').strip()
clock=clock.strip(string.punctuation+' ')
print(clock)
print(clockspeed)
testdict={}
testdict[clock]=float(clockspeed)
print(testdict)

Turbo Speed
5.7
{'Turbo Speed': 5.7}


In [43]:
import string

In [50]:
puncA="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [51]:
string.punctuation+puncA

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '

In [54]:
test = 'asghsdhsad3.5 Ghz'
test.strip(string.punctuation+puncA)

'3.5'

In [ ]:
pprint.pprint(cpu.find())
# print(cpu.find()[0]['specsID'])
cpuList = cpu.find()
cpuIDList = []
for i in cpuList:
    cpuIDList.append(i['specsID'])
#     print(i['specsID'])
    
cpuIDList=set(cpuIDList)
# print(len(cpuIDList))
print(cpuIDList)
for i in cpuIDList:
    print(i)